In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load dataset
file_path = "Telco_customer_churn.xlsx" # Replace with your file path
df = pd.read_excel(file_path)

# Preprocessing
# Drop irrelevant columns
columns_to_drop = ['CustomerID', 'Country', 'State', 'City', 'Lat Long', 'Churn Reason', 'Churn Score']
df = df.drop(columns=columns_to_drop, errors='ignore')

# Convert 'Churn Label' to numeric (0 for 'No', 1 for 'Yes')
df['Churn Label'] = df['Churn Label'].apply(lambda x: 1 if x == 'Yes' else 0)

# Convert Total Charges to numeric
df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors='coerce')
df['Total Charges'] = df['Total Charges'].fillna(0)

# Encode categorical variables
binary_columns = ['Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Paperless Billing']
label_encoders = {}
for col in binary_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# One-hot encode multi-class columns
multi_class_columns = ['Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup',
                       'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies',
                       'Contract', 'Payment Method']
df = pd.get_dummies(df, columns=multi_class_columns, drop_first=True)

# Define features and target
X = df.drop(columns=['Churn Label', 'Churn Value'], errors='ignore')  # Drop both Churn Label and Churn Value
y = df['Churn Label']  # Use Churn Label as the target variable

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Scale features for Logistic Regression
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled_scaled, y_resampled, test_size=0.3, random_state=42)

# Define models to compare
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    "SVM (Linear Kernel)": SVC(kernel='linear', probability=True, random_state=42)
}

# Compare models
results = []
for model_name, model in models.items():
    # Train model
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

    # Evaluate performance
    accuracy = accuracy_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else "N/A"
    report = classification_report(y_test, y_pred, output_dict=True)

    # Handle missing classes in report
    precision_churn = report['1']['precision'] if '1' in report else 0.0
    recall_churn = report['1']['recall'] if '1' in report else 0.0
    f1_score_churn = report['1']['f1-score'] if '1' in report else 0.0

    # Store results
    results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "AUC-ROC": auc_roc,
        "Precision (Churn)": precision_churn,
        "Recall (Churn)": recall_churn,
        "F1-Score (Churn)": f1_score_churn
    })

# Display results
results_df = pd.DataFrame(results).sort_values(by="AUC-ROC", ascending=False)
print(results_df)

# Save results to an Excel file
results_df.to_excel("model_comparison_results.xlsx", index=False)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:37:15] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warni

[LightGBM] [Info] Number of positive: 3624, number of negative: 3619
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002865 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1685
[LightGBM] [Info] Number of data points in the train set: 7243, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500345 -> initscore=0.001381
[LightGBM] [Info] Start training from score 0.001381


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


                 Model  Accuracy   AUC-ROC  Precision (Churn)  Recall (Churn)  \
4             LightGBM  0.856039  0.939742           0.847950        0.867097   
1        Random Forest  0.860548  0.938903           0.845820        0.881290   
3              XGBoost  0.861836  0.938433           0.859987        0.863871   
5  SVM (Linear Kernel)  0.825443  0.916343           0.836898        0.807742   
0  Logistic Regression  0.822544  0.916022           0.828402        0.812903   
2        Decision Tree  0.803543  0.803543           0.802835        0.803871   

   F1-Score (Churn)  
4          0.857416  
1          0.863191  
3          0.861925  
5          0.822062  
0          0.820580  
2          0.803353  


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import shap

# Load dataset
file_path = "Telco_customer_churn.xlsx"  # Replace with your file path
df = pd.read_excel(file_path)

# Preprocessing
# Drop irrelevant columns
columns_to_drop = ['CustomerID', 'Country', 'State', 'City', 'Lat Long', 'Churn Reason', 'Churn Score']
df = df.drop(columns=columns_to_drop, errors='ignore')

# Convert 'Churn Label' to numeric
df['Churn Label'] = df['Churn Label'].apply(lambda x: 1 if x == 'Yes' else 0)

# Convert Total Charges to numeric
df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors='coerce')
df['Total Charges'] = df['Total Charges'].fillna(0)

# Encode categorical variables
binary_columns = ['Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Paperless Billing']
label_encoders = {}
for col in binary_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

multi_class_columns = ['Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup',
                       'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies',
                       'Contract', 'Payment Method']
df = pd.get_dummies(df, columns=multi_class_columns, drop_first=True)

# Define features and target
X = df.drop(columns=['Churn Label', 'Churn Value'], errors='ignore')
y = df['Churn Label']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model Feature Importance Extraction
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

feature_importance = pd.DataFrame({'Feature': X.columns})

for model_name, model in models.items():
    model.fit(X_train_scaled, y_train)
    if hasattr(model, "feature_importances_"):
        # Tree-based models
        feature_importance[model_name] = model.feature_importances_
    elif hasattr(model, "coef_"):
        # Logistic Regression
        feature_importance[model_name] = np.abs(model.coef_[0])

# Average Importance
feature_importance['AverageImportance'] = feature_importance.iloc[:, 1:].mean(axis=1)
feature_importance = feature_importance.sort_values(by='AverageImportance', ascending=False)

print("Top Features Based on Average Importance:")
print(feature_importance.head(10))

# Recursive Feature Elimination (RFE) with Logistic Regression
rfe = RFE(estimator=LogisticRegression(max_iter=1000, random_state=42), n_features_to_select=10)
rfe.fit(X_train_scaled, y_train)
rfe_features = X.columns[rfe.support_]
print("\nTop Features Identified by RFE:")
print(rfe_features)

# SHAP Values for XGBoost
explainer = shap.Explainer(models['XGBoost'], X_train_scaled)
shap_values = explainer(X_train_scaled)
shap.summary_plot(shap_values, X_train, plot_type="bar")

# Final Output
final_features = feature_importance.head(10)['Feature'].tolist()
print("\nFinal Selected Features for All Models:")
print(final_features)


FileNotFoundError: [Errno 2] No such file or directory: 'Telco_customer_churn.xlsx'